# PyTorch 2 Export

```{contents}
```

## torch.export 101

The main idea of `torch.export` is that it translates an Eager Mode PyTorch model into a graph-based intermediate representation called *Export IR*. This allows compiler backends to take this IR and further transform and optimize it for a target device. A general overview of the process is shown in the figure [below](torchexport).

:::{figure-md} torchexport
<img src="compilation.png" alt="torch.export" width=70%>

PyTorch 2 Export
:::

This IR needs to fulfill a couple of properties for it to be useful to compilers. For example:
1. Operators have to be general enough for backends to notice patterns and optimize them: Many runtimes have specialized kernels  for common operators like convolutions or even more complex ones like a `conv2 + batchnorm` (operator fusion). If the IR reduces all operators to sums, products and views, noticing these patterns becomes too hard.
2. The number of operators has to be small enough for the backend to implement all of them. 
3. Operators have to be functional, that is, without side effects. For example: If two functions read and modify the same parameters, the order of execution matters and the compiler has to be careful when parallelizing them.

Notice that properties 1 and 2 are in conflict with each other. The more operators we have, the more expressive the IR is, but the harder it is to implement all of them. This is a trade-off that the PyTorch team has to balance. 


For now, let's get some practical intuition with an example.

Let's use the following simple network:

## Playing with torch.export

In [3]:
import torch.nn as nn
import torch
import torch.nn.functional as F

In [19]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 6, 5)
        self.fc1 = nn.Linear(6 * 14 * 14, 10)
        self.fc2 = nn.Linear(6 * 14 * 14, 10)
        self.register_buffer("mask", torch.randn(6, 14, 14) > 0.5)

    def forward(self, x: torch.Tensor):
        x = self.conv(x)
        x = F.relu(x)
        x = F.max_pool2d(x, (2, 2))
        x = x * self.mask
        x = torch.flatten(x, 1)
        y = self.fc1(x)
        z = self.fc2(x)
        y = F.relu(y) 
        z = F.relu(z)
        return z, y


In [20]:
x = torch.randn(1, 3, 32, 32) 
ep: torch.export.ExportedProgram = torch.export.export(SimpleNet(), (x,))

In [ ]:
ep.p

In [21]:
torch.export.save(ep, "simple_net.pt2")